In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('car_evaluation.csv',header=None)
df.head()

,0,1,2,3,4,5,6
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       1728 non-null   object
 1   1       1728 non-null   object
 2   2       1728 non-null   object
 3   3       1728 non-null   object
 4   4       1728 non-null   object
 5   5       1728 non-null   object
 6   6       1728 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


In [4]:
df.describe()

,0,1,2,3,4,5,6
count,1728,1728,1728,1728,1728,1728,1728
unique,4,4,4,3,3,3,4
top,low,low,2,2,big,low,unacc
freq,432,432,432,576,576,576,1210


In [5]:
col_names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
df.columns = col_names

In [6]:
df.head()

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [7]:
df.nunique()

buying      4
maint       4
doors       4
persons     3
lug_boot    3
safety      3
class       4
dtype: int64

In [8]:
for i in df.columns:
    print(df[i].unique())

['vhigh' 'high' 'med' 'low']
['vhigh' 'high' 'med' 'low']
['2' '3' '4' '5more']
['2' '4' 'more']
['small' 'med' 'big']
['low' 'med' 'high']
['unacc' 'acc' 'vgood' 'good']


In [9]:
#pip install --user pandas-profiling

In [10]:
from pandas_profiling import ProfileReport

In [11]:
profile = ProfileReport(df,title = 'EDA report',explorative=True)
profile.to_widgets()
profile.to_file("output.html")

Summarize dataset:   0%|          | 0/21 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
import category_encoders as ce

In [13]:
encoder = ce.OrdinalEncoder()
df1 = encoder.fit_transform(df)

In [14]:
df1.head()

,buying,maint,doors,persons,lug_boot,safety,class
0,1,1,1,1,1,1,1
1,1,1,1,1,1,2,1
2,1,1,1,1,1,3,1
3,1,1,1,1,2,1,1
4,1,1,1,1,2,2,1


In [15]:
for i in df1.columns:
    print(df1[i].unique())

[1 2 3 4]
[1 2 3 4]
[1 2 3 4]
[1 2 3]
[1 2 3]
[1 2 3]
[1 2 3 4]


In [17]:
X = df1.drop('class',axis=1)
y = df1['class']

In [19]:
X.shape,y.shape

((1728, 6), (1728,))

In [16]:
from sklearn.model_selection import train_test_split

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [21]:
from sklearn.ensemble import RandomForestClassifier

In [24]:
rfc = RandomForestClassifier(random_state=42)
rfc.fit(X_train,y_train)
y_pred = rfc.predict(X_test)

In [25]:
from sklearn.metrics import accuracy_score

In [26]:
accuracy_score(y_test,y_pred)

0.9633911368015414

In [30]:
for i in zip(X_train.columns,rfc.feature_importances_):
    print(i)

('buying', 0.15323260331662295)
('maint', 0.15816215152759916)
('doors', 0.060025863011928395)
('persons', 0.2364815388058295)
('lug_boot', 0.09476639869028768)
('safety', 0.29733144464773237)


In [31]:
X1_train = X_train.drop(['doors','lug_boot'],axis= 1)

In [32]:
X1_test = X_test.drop(['doors','lug_boot'],axis= 1)

In [33]:
rfc1 = RandomForestClassifier(random_state=42)
rfc1.fit(X1_train,y_train)
y1_pred = rfc1.predict(X1_test)
accuracy_score(y_test,y1_pred)

0.8554913294797688

In [34]:
X2_train = X_train.drop(['lug_boot'],axis= 1)
X2_test = X_test.drop(['lug_boot'],axis= 1)

In [36]:
rfc2 = RandomForestClassifier(random_state=42)
rfc2.fit(X2_train,y_train)
y2_pred = rfc2.predict(X2_test)
accuracy_score(y_test,y2_pred)

0.8208092485549133

In [37]:
X3_train = X_train.drop(['doors'],axis= 1)
X3_test = X_test.drop(['doors'],axis= 1)

In [38]:
rfc3 = RandomForestClassifier(random_state=42)
rfc3.fit(X3_train,y_train)
y3_pred = rfc3.predict(X3_test)
accuracy_score(y_test,y3_pred)

0.9344894026974951

In [39]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.99      0.99      0.99       358
           2       0.93      0.92      0.92       118
           3       0.91      0.88      0.89        24
           4       0.76      0.84      0.80        19

    accuracy                           0.96       519
   macro avg       0.90      0.91      0.90       519
weighted avg       0.96      0.96      0.96       519



In [44]:
rfc4 = RandomForestClassifier(n_estimators=80,random_state=42)
rfc4.fit(X_train,y_train)
y4_pred = rfc4.predict(X_test)
accuracy_score(y_test,y4_pred)

0.9653179190751445